In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math
import time
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.utils import shuffle

In [2]:
# Declare Cross-Fold Validation Split / Model Params
N_SPLIT = 10
EPOCHS = 650
LAYERSIZE = 40
AUTO_DIM = 80

In [3]:
# Read Data
xdf = pd.read_csv('ML_inputs_tear.csv')
ydf = pd.read_csv('U_tear.csv')

# Adding Constant Parameters to X data
xdf.insert(0, 'c', 6)
xdf.insert(1, 'D', .05)
xdf.insert(2, 'k1', 15)
xdf.insert(3, 'k2', 15)
xdf.insert(4, 'N', 2)
xdf.insert(5, 'kappa_1', .25)
xdf.insert(6, 'kappa_2', .25)
xdf.insert(7, 'kappa_3', .25)

In [4]:
# Convert to Array
x = xdf.to_numpy()
y = ydf.T.to_numpy()

In [5]:
class Autoencoder(tf.keras.models.Model):
    def __init__(self, dim):
        super(Autoencoder, self).__init__()
        self.dim = dim
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Dense(len(y[0])),
            tf.keras.layers.Dense(500, 'relu'),
            tf.keras.layers.Dense(500, 'relu'),
            tf.keras.layers.Dense(dim)
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(500, 'relu'),
            tf.keras.layers.Dense(500, 'relu'),
            tf.keras.layers.Dense(len(y[0]))
        ])
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [6]:
autoencoder = Autoencoder(AUTO_DIM)
autoencoder.compile(optimizer = 'adamax', loss = 'mse')

In [7]:
start = time.time()
autoencoder.fit(y, y, epochs = 500)
end = time.time()

auto_train_time = (end-start)

Epoch 1/500
8/8 [==============================] - 1s 6ms/step - loss: 0.1960
Epoch 2/500
8/8 [==============================] - 0s 6ms/step - loss: 0.0450
Epoch 3/500
8/8 [==============================] - 0s 6ms/step - loss: 0.0189
Epoch 4/500
8/8 [==============================] - 0s 6ms/step - loss: 0.0110
Epoch 5/500
8/8 [==============================] - 0s 6ms/step - loss: 0.0088
Epoch 6/500
8/8 [==============================] - 0s 6ms/step - loss: 0.0076
Epoch 7/500
8/8 [==============================] - 0s 6ms/step - loss: 0.0066
Epoch 8/500
8/8 [==============================] - 0s 5ms/step - loss: 0.0059
Epoch 9/500
8/8 [==============================] - 0s 5ms/step - loss: 0.0048
Epoch 10/500
8/8 [==============================] - 0s 5ms/step - loss: 0.0039
Epoch 11/500
8/8 [==============================] - 0s 5ms/step - loss: 0.0029
Epoch 12/500
8/8 [==============================] - 0s 6ms/step - loss: 0.0026
Epoch 13/500
8/8 [==============================] - 0s 6ms/st

In [8]:
encoded = autoencoder.encoder.predict(y)
encoded.shape

(240, 80)

In [9]:
#Define Model

def create_model():
    
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(12),
        tf.keras.layers.Dense(LAYERSIZE, 'sigmoid'),
        tf.keras.layers.Dense(LAYERSIZE, 'sigmoid'),
        tf.keras.layers.Dense(AUTO_DIM)
    ])
    
    model.compile(optimizer = 'adam', loss = 'mse')
    
    return model

In [10]:
train_time = []
annmodels = []
test_val = []
acc = []
train_val = []

for train_index, test_index in model_selection.KFold(N_SPLIT, shuffle=True).split(x):
    xtrain, xtest = x[train_index], x[test_index]
    ytrain, ytest = encoded[train_index], encoded[test_index]
    
    start = time.time()
    model = create_model()
    model.fit(xtrain, ytrain, epochs = EPOCHS, verbose = 2)
    end = time.time()
    
    train_time.append(end-start)
    
    annmodels.append(model)
    test_val.append((xtest, ytest))
    train_val.append((xtrain, ytrain))
    acc.append(model.evaluate(xtest, ytest))

Epoch 1/650
7/7 - 0s - loss: 25.5691
Epoch 2/650
7/7 - 0s - loss: 24.2148
Epoch 3/650
7/7 - 0s - loss: 22.9399
Epoch 4/650
7/7 - 0s - loss: 21.6506
Epoch 5/650
7/7 - 0s - loss: 20.3267
Epoch 6/650
7/7 - 0s - loss: 18.9671
Epoch 7/650
7/7 - 0s - loss: 17.5968
Epoch 8/650
7/7 - 0s - loss: 16.2102
Epoch 9/650
7/7 - 0s - loss: 14.8504
Epoch 10/650
7/7 - 0s - loss: 13.5399
Epoch 11/650
7/7 - 0s - loss: 12.3076
Epoch 12/650
7/7 - 0s - loss: 11.1750
Epoch 13/650
7/7 - 0s - loss: 10.0992
Epoch 14/650
7/7 - 0s - loss: 9.1674
Epoch 15/650
7/7 - 0s - loss: 8.3626
Epoch 16/650
7/7 - 0s - loss: 7.6885
Epoch 17/650
7/7 - 0s - loss: 7.0956
Epoch 18/650
7/7 - 0s - loss: 6.5832
Epoch 19/650
7/7 - 0s - loss: 6.1452
Epoch 20/650
7/7 - 0s - loss: 5.7665
Epoch 21/650
7/7 - 0s - loss: 5.4469
Epoch 22/650
7/7 - 0s - loss: 5.1628
Epoch 23/650
7/7 - 0s - loss: 4.9226
Epoch 24/650
7/7 - 0s - loss: 4.7224
Epoch 25/650
7/7 - 0s - loss: 4.5405
Epoch 26/650
7/7 - 0s - loss: 4.3776
Epoch 27/650
7/7 - 0s - loss: 4.24

In [11]:
# Timing Calculations
modelTime = []

for model in annmodels:
    
    # Perform 50 timed predictions, average result
    tmpTime = []
    for i in range(50):
        start = time.time()
        pred = model.predict(x)
        auto_pred = autoencoder.decoder.predict(pred)
        end = time.time()
        tmpTime.append(end - start)
        
    modelTime.append(np.mean(tmpTime))

In [12]:
# Error Calculations
modelError = []

for model in annmodels:
    # find the relative frobenius error of the prediction
    pred = model.predict(x)
    auto_pred = autoencoder.decoder.predict(pred)
    
    toterr = 0
    for i in range(len(x)):
        Norm1 = np.linalg.norm(y[i] - auto_pred[i], 2)
        Norm2 = np.linalg.norm(y[i], 2)
        toterr = toterr + ((Norm1**2)/(Norm2**2))
        
    modelError.append(math.sqrt(toterr))

In [13]:
export = pd.DataFrame({'offline time': train_time,'online time': modelTime, 'error': modelError})

In [14]:
export

,offline time,online time,error
0,4.647209,0.069298,0.353105
1,4.868989,0.075624,0.369713
2,4.818835,0.067846,0.443558
3,4.741733,0.068229,0.382096
4,4.868367,0.066681,0.380527
5,4.797615,0.069561,0.364571
6,4.833052,0.068066,0.403021
7,4.697266,0.077392,0.366526
8,4.680925,0.069988,0.365165
9,4.666751,0.070302,0.363349


In [15]:
auto_train_time

23.829305171966553